In [18]:
%load_ext autoreload
%autoreload 2
# Set-up: Import numpy and assign GPU
from collections import defaultdict
import os
import numpy as np

from gpu_utils import restrict_GPU_pytorch
restrict_GPU_pytorch('2')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using GPU:2


In [2]:
# Load Amazon WILDS pre-trained model
import sys
import pickle
import numpy as np
import pandas as pd

import torch
import torchvision.transforms as transforms

import argparse
import pdb

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds.common.grouper import CombinatorialGrouper

sys.path.insert(0, './wilds/examples/')
from transforms import initialize_transform, getBertTokenizer
from algorithms.initializer import initialize_algorithm


In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
#config_dict = pickle.load(open('amazon_config.txt', 'rb'))

infile = open('civilcomments_config.txt','rb')
new_dict = vars(pickle.load(infile))
infile.close() 

new_dict['transform'] = 'nlp_ocr'

config = Namespace(**new_dict)
config.algorithm = 'ERM'


In [4]:
new_dict

{'dataset': 'civilcomments',
 'algorithm': 'ERM',
 'root_dir': 'data',
 'split_scheme': 'official',
 'dataset_kwargs': {},
 'download': False,
 'frac': 1.0,
 'version': None,
 'loader_kwargs': {'num_workers': 1, 'pin_memory': True},
 'train_loader': 'standard',
 'uniform_over_groups': False,
 'distinct_groups': None,
 'n_groups_per_batch': 4,
 'batch_size': 16,
 'eval_loader': 'standard',
 'model': 'distilbert-base-uncased',
 'model_kwargs': {},
 'transform': 'nlp_ocr',
 'target_resolution': None,
 'resize_scale': None,
 'max_token_length': 300,
 'loss_function': 'cross_entropy',
 'loss_kwargs': {},
 'groupby_fields': ['black', 'y'],
 'group_dro_step_size': None,
 'coral_penalty_weight': 10.0,
 'irm_lambda': 1.0,
 'irm_penalty_anneal_iters': None,
 'algo_log_metric': 'accuracy',
 'val_metric': 'acc_wg',
 'val_metric_decreasing': False,
 'n_epochs': 5,
 'optimizer': 'AdamW',
 'lr': 1e-05,
 'weight_decay': 0.01,
 'max_grad_norm': 1.0,
 'optimizer_kwargs': {},
 'scheduler': 'linear_schedu

In [5]:

full_dataset = get_dataset(dataset='civilcomments', download=False, root_dir = './wilds/data')

print(type(full_dataset))

<class 'wilds.datasets.civilcomments_dataset.CivilCommentsDataset'>


In [6]:
train_transform = initialize_transform(transform_name=config.transform,
                                      config=config,
                                      dataset=full_dataset,
                                      is_training=True)
eval_transform = initialize_transform(transform_name=config.transform,
                                      config=config,
                                      dataset=full_dataset,
                                      is_training=False)

In [7]:
train_data = full_dataset.get_subset('train', transform=train_transform)
train_loader = get_train_loader('standard', train_data, batch_size=64)

train_grouper = CombinatorialGrouper(dataset=full_dataset, groupby_fields=config.groupby_fields)

In [8]:
train_data[1]

Blame men.  There's always an excuse to blame men for the failures of women.
Blame men. There ' 8 always an excuse to blame men for the failokes of wumen.


(tensor([[  101,     1],
         [ 7499,     1],
         [ 2273,     1],
         [ 1012,     1],
         [ 2045,     1],
         [ 1005,     1],
         [ 1022,     1],
         [ 2467,     1],
         [ 2019,     1],
         [ 8016,     1],
         [ 2000,     1],
         [ 7499,     1],
         [ 2273,     1],
         [ 2005,     1],
         [ 1996,     1],
         [ 8246,     1],
         [11045,     1],
         [ 2015,     1],
         [ 1997,     1],
         [ 8814,     1],
         [ 3549,     1],
         [ 1012,     1],
         [  102,     1],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],


In [9]:
datasets = defaultdict(dict)
for split in full_dataset.split_dict.keys():
    if split=='train':
        transform = train_transform
        verbose = True
    elif split == 'val':
        transform = eval_transform
        verbose = True
    else:
        transform = eval_transform
        verbose = False
    # Get subset
    datasets[split]['dataset'] = full_dataset.get_subset(
        split,
        frac=config.frac,
        transform=transform)

    if split == 'train':
        datasets[split]['loader'] = get_train_loader(
            loader=config.train_loader,
            dataset=datasets[split]['dataset'],
            batch_size=config.batch_size,
            uniform_over_groups=config.uniform_over_groups,
            grouper=train_grouper,
            distinct_groups=config.distinct_groups,
            n_groups_per_batch=config.n_groups_per_batch,
        **config.loader_kwargs)
    else:
        datasets[split]['loader'] = get_eval_loader(
            loader=config.eval_loader,
            dataset=datasets[split]['dataset'],
            grouper=train_grouper,
            batch_size=config.batch_size,
            **config.loader_kwargs)

# Set fields
datasets[split]['split'] = split
datasets[split]['name'] = full_dataset.split_names[split]
datasets[split]['verbose'] = verbose

# Loggers
# datasets[split]['eval_logger'] = BatchLogger(
#     os.path.join(config.log_dir, f'{split}_eval.csv'), mode=mode, use_wandb=(config.use_wandb and verbose))
# datasets[split]['algo_logger'] = BatchLogger(
#     os.path.join(config.log_dir, f'{split}_algo.csv'), mode=mode, use_wandb=(config.use_wandb and verbose))

In [10]:
model = initialize_algorithm(config, datasets, train_grouper)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [19]:
model.model.cuda()

DistilBertClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [20]:
it = iter(train_loader)
batch = next(it)


HANNITY?!? Comrade RTH you're too funny! There will be no major announcement. Hannity is only trying to distract his low information, low IQ followers from the disaster that is Trump and the Republicans that are "running" Washington now. So you are calling a true American hero "McLame" now because he had the courage of his convictions and went against your Dear Leader? Only a brainwashed sycophant would make an insane comment like that.....Dasvidaniya Comrade RTH.....#MRGA #itmfa
HANNITY? !? C0mkade RTH you ' re too funny! There wil1 be no major announcement. Hannity is only trying to distract his low infukmati0n, low IQ followers from the disaster that is Trump and the Republicans that are " running " Washington now. So you are calling a true American hero " McLame " n0w because he had the courage of hi8 convictions and went against your Dear Leader? Only a brainwashed 8ycuphant would make an insane comment like that. .. .. Dasvidaniya Comrade RTH. .. .. # MRGA # itmfa
"If you know an

In [29]:
len(batch[1])
batch[2]

# batch[0] is array of input features
# batch[1] is array of labels
# batch[2] is array of metadata

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 1, 0, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [23]:
model.model(torch.unsqueeze(batch[0][0], 0).cuda())

# Outputting before transforming to measure toxicity

# Apply softmax to normalize and sum to 1

# Save predictions while iterating through samples into .npy or pickl (np.save)

tensor([[0.1647, 0.0575]], device='cuda:0', grad_fn=<AddmmBackward>)

In [32]:
datasets[split]

{'dataset': <wilds.datasets.wilds_dataset.WILDSSubset at 0x7fa531852a58>,
 'loader': <torch.utils.data.dataloader.DataLoader at 0x7fa531852a90>,
 'split': 'test',
 'name': 'Test',
 'verbose': False}

In [33]:
datasets['val']['loader']